# Fugacity of CH4 and N2 mixture from example 15.2 of [0529]

## Using cubic EOS

In [30]:
#classes
from eos import EOS
from alpha_r_cubic import AlphaRCubic
from mixture_fluid import MixtureFluid
from eos_mixture import EOSMixture

#objects
from purefluid import N2, CH4
from cp0 import N2_cp0_NIST, CH4_cp0_NIST

#other
from symbols import *
from mixture_rules import one_fluid_theory

############################################## EOS for pure fluids
# create PR EOS for both N2 and CH4
PR_expr = sp.log(abs(1/rho/(1/rho-b))) + 1/(R*T)*a/(4*b)*2**0.5*sp.log(abs((1/rho-b*(2**0.5-1))/(1/rho+b*(2**0.5+1))))
a_c_expr = 0.4572*R**2*T_c**2/P_c
alpha_T_expr = (1+(0.37464+1.5422*omega - 0.2699*omega**2)*(1-(T/T_c)**0.5))**2
b_expr = 0.07779607*R*T_c/P_c

N2_PR = EOS("N2_PR", N2, N2_cp0_NIST, AlphaRCubic, alpha_r_expr=PR_expr, a_c_expr=a_c_expr, alpha_T_expr=alpha_T_expr, b_expr=b_expr)
CH4_PR = EOS("CH4_PR", CH4, CH4_cp0_NIST, AlphaRCubic, alpha_r_expr=PR_expr, a_c_expr=a_c_expr, alpha_T_expr=alpha_T_expr, b_expr=b_expr)

############################################## EOS for pure mixture
#create mixture EOS
mix = MixtureFluid(N2, CH4)

mix_PR = EOSMixture("PR_N2_CH4", mix, AlphaRCubic, N2_PR, CH4_PR, mixture_rule=one_fluid_theory, delta=0, k_ij=[[0,0],[0,0]])


############################################## Example settings
# test against example 15.2 of chemical engineering book
pressure = 0.4119*1e6
temperature = 100
z = [0.958, 1-0.958]
density = 0.4119*1e6/(0.9059*R*temperature)

############################################## a, b and pressure
a = mix_PR.alpha_r.a.subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])
b = mix_PR.alpha_r.b.subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])
pressure = mix_PR.pressure.subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])

print(f"a = {a}")
print(f"b = {b}")
print(f"pressure  = {pressure}")

print("")

############################################## Fugacity coefficients
phi_i = mix_PR.fugacity_coefficients

fugacity_coefficients = [sp.log(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"logarithm of fugacity coefficients = {fugacity_coefficients}")

# fugacity coefficients = [-0.0879702573569293, -0.165118060729465]


a = 0.167784476819040
b = 0.0000241409521005300
pressure  = 411874.172344260

logarithm of fugacity coefficients = [-0.0876164978134070, -0.165991189041858]


## Using Helmholtz EOS without F argument

In [31]:
#classes
from eos import EOS
from alpha_r_helmholtz import AlphaRHelmholtz
from mixture_fluid import MixtureFluid
from eos_mixture import EOSMixture

#objects
from purefluid import N2, CH4
from cp0 import N2_cp0_NIST, CH4_cp0_NIST
from presets import alpha_r_0543, alpha_r_0542

#other
from symbols import *

############################################## EOS for pure fluids
#create pure fluid EOS for both fluids
CH4_0543 = EOS("CH4_0543", CH4, CH4_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0543())
N2_0542 = EOS("N2_0542", N2, N2_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0542())

############################################## EOS for pure mixture
#create mixture EOS
mix = MixtureFluid(N2, CH4)

mix_EOS = EOSMixture("N2_0542_CH4_0543", mix, AlphaRHelmholtz, N2_0542, CH4_0543)


############################################## Example settings
# test against example 15.2 of chemical engineering book
pressure = 0.4119*1e6
temperature = 100
z = [0.958, 1-0.958]
density = 0.4119*1e6/(0.9059*R*temperature)

##############################################pressure
pressure = mix_EOS.pressure.subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])

print(f"pressure  = {pressure}")

print("")

############################################## Fugacity coefficients
phi_i = mix_EOS.fugacity_coefficients

fugacity_coefficients = [sp.log(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"logarithm of fugacity coefficients = {fugacity_coefficients}")



pressure  = 413415.464630590

logarithm of fugacity coefficients = [-0.0834120191408151, -0.139510681675204]


## Using Helmholtz EOS with F argument

In [3]:
#classes
from eos import EOS
from alpha_r_helmholtz import AlphaRHelmholtz
from mixture_fluid import MixtureFluid
from eos_mixture import EOSMixture

#objects
from purefluid import N2, CH4
from cp0 import N2_cp0_NIST, CH4_cp0_NIST
from presets import alpha_r_0543, alpha_r_0542

#other
from symbols import *

############################################## EOS for pure fluids
#create pure fluid EOS for both fluids
CH4_0543 = EOS("CH4_0543", CH4, CH4_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0543())
N2_0542 = EOS("N2_0542", N2, N2_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0542())

############################################## EOS for mixture
#create mixture EOS
mix = MixtureFluid(N2, CH4)

#coefficients
k_pol = 2
k_exp = 7

n_ij = [-0.98038985517335e-2, 0.42487270143005e-3, -0.34800214576142e-1, -0.13333813013896, -0.11993694974627e-1, 0.69243379775168e-1, -0.31022508148249, 0.24495491753226, 0.22369816716981]
d_ij = [1, 4, 1, 2, 2, 2, 2, 2, 3]
t_ij = [0, 1.85, 7.85, 5.4, 0, 0.75, 2.8, 4.45, 4.25]

eta_ij = [0, 0, 1, 1, 0.25, 0, 0, 0, 0]
epsilon_ij = [0, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
beta_ij =[0, 0, 1, 1, 2.5, 3, 3, 3, 3]
gamma_ij = [0, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

# binary coefficients
beta_V =[[1, 0.998721377],[1/0.998721377, 1]]
gamma_V = [[1, 1.013950311], [1.013950311, 1]]
beta_T = [[1, 0.998098830],[1/0.998098830, 1]]
gamma_T = [[1, 0.979273013], [0.979273013, 1]]

mix_EOS = EOSMixture("N2_0542_CH4_0543", mix, AlphaRHelmholtz, N2_0542, CH4_0543,
                     beta_T=beta_T, gamma_T=gamma_T, beta_V=beta_V, gamma_V=gamma_V,
                     F=0, n_ij=n_ij, d_ij=d_ij, t_ij=t_ij, eta_ij=eta_ij, epsilon_ij=epsilon_ij, beta_ij=beta_ij, gamma_ij=gamma_ij,
                     k_pol=k_pol, k_exp=k_exp )


############################################## Example settings
# test against example 15.2 of chemical engineering book
pressure = 0.4119*1e6
temperature = 100
z = [0.958, 1-0.958]
density = 0.4119*1e6/(0.9059*R*temperature)

############################################## Pressure
pressure = mix_EOS.pressure.subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])

print(f"pressure  = {pressure}")

print("")

############################################## Fugacity coefficients
phi_i = mix_EOS.fugacity_coefficients

fugacity_coefficients = [sp.log(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"logarithm of fugacity coefficients = {fugacity_coefficients}")

# logarithm of fugacity coefficients = [-0.0876164978134070, -0.165991189041858]    with cubic EOS

# logarithm of fugacity coefficients = [-0.0831799070525770, -0.149228637763201]   with F = 1
# logarithm of fugacity coefficients = [-0.0832953768783173, -0.133129793184776]    with F = 0

pressure  = 413624.475380347

logarithm of fugacity coefficients = [-0.0832953768783173, -0.133129793184776]


# Fugacity of NH3 and H2 mixture

## Using cubic EOS

In [6]:

# classes
from eos import EOS
from eos_mixture import EOSMixture
from  alpha_r_cubic import AlphaRCubic
from mixture_fluid import MixtureFluid

# objects
from purefluid import NH3, H2
from cp0 import NH3_cp0_NIST, H2_cp0_NIST 


# algorithms and other functions and variables
from symbols import *
from mixture_rules import one_fluid_theory
from algorithms import MSRK_coefficients, calc_density_coefficients


############################################## EOS for pure fluids
#expressions for SRK
MSRK = {
    "alpha_r_expr": sp.log(abs((1/rho/(1/rho-b)))) + 1/(R*T)*a/b*sp.log(abs(1/rho/(1/rho+b))),
    "a_c_expr": 0.42748*R**2*T_c**2/P_c,
    "alpha_T_expr": (1+ (0.48503 + 1.5571*omega - 0.15613*omega**2)*(1-(T/T_c)**0.5))**2,
    "b_expr": 0.08664*R*T_c/P_c
}

#create pure fluid EOS from MSRK
MSRK_NH3 = EOS("SRK_NH3", NH3, NH3_cp0_NIST, AlphaRCubic, **MSRK)
MSRK_H2 = EOS("SRK_H2", H2, H2_cp0_NIST, AlphaRCubic, **MSRK)

############################################## EOS for pure mixture
#create mixture EOS
mix = MixtureFluid(H2, NH3)

mix_EOS = EOSMixture("MSRK_H2_NH3", mix, AlphaRCubic, MSRK_H2, MSRK_NH3, mixture_rule=one_fluid_theory)

####################################### Temperature and pressure and composition settings
temperature = 298
pressure = 10e5
z = [0.2,1-0.2]

####################################### density
densities = calc_density_coefficients(mix_EOS, pressure, temperature, *z, MSRK_coefficients)

#pick a value of density
density = densities[-1]
print(f"all densities = {densities}")
print(f"density = {density}")
####################################### fugacity
phi_i = mix_EOS.fugacity_coefficients

fugacity_coefficients = [sp.log(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"logarithm of fugacity coefficients = {fugacity_coefficients}")




all densities = [  426.26668635 11397.83905294 22714.5155116 ]
density = 22714.51551159571
logarithm of fugacity coefficients = [5.12514230492424, 0.118585440888818]


## Using Helmholtz EOS, [0290] + [0313]

In [21]:
# classes
from eos import EOS
from eos_mixture import EOSMixture
from  alpha_r_helmholtz import AlphaRHelmholtz
from mixture_fluid import MixtureFluid

# objects
from purefluid import NH3, H2
from cp0 import NH3_cp0_NIST, H2_cp0_NIST 

# algorithms and other functions and variables
from symbols import *
from other_functions import multi_root
from presets import alpha_r_0290, alpha_r_0313



####################################### EOS
#create pure fluid EOS for both fluids
NH3_0290 = EOS("NH3_0290", NH3, NH3_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0290())
H2_0313 = EOS("H2_0313", H2, H2_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0313())

# create mixture and mixture EOS (hydrogen must be the first fluid)
mix = MixtureFluid(H2, NH3)

mix_EOS = EOSMixture("H2_0313_NH3_0290", mix, AlphaRHelmholtz, H2_0313, NH3_0290)

####################################### Temperature and pressure and composition settings
temperature = 298
pressure = 10e5
z = [0,1]

####################################### density
densities = multi_root(mix_EOS.pressure_equation, [1,5], (pressure, temperature, *z), n=1e5)

#pick a value of density
density = densities[-1]
print(f"all densities = {densities}")
print(f"density = {density} [mol/m3]")
print(f"density = {density*NH3.M} [kg/m3]")

####################################### fugacity
phi_i = mix_EOS.fugacity_coefficients

fugacity_coefficients = [(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"fugacity coefficients = {fugacity_coefficients}")



all densities = [  456.99390222  3941.95596019 35417.67122083]
density = 35417.6712208268 [mol/m3]
density = 603.1813580797152 [kg/m3]
fugacity coefficients = [148805.662252930, 0.895485336648989]


## Using Helmholtz EOS, [0300] + [0313]

In [13]:
# classes
from eos import EOS
from eos_mixture import EOSMixture
from  alpha_r_helmholtz import AlphaRHelmholtz
from mixture_fluid import MixtureFluid

# objects
from purefluid import NH3, H2
from cp0 import NH3_cp0_NIST, H2_cp0_NIST 

# algorithms and other functions and variables
from symbols import *
from presets import alpha_r_0300, alpha_r_0313
from other_functions import multi_root

####################################### EOS
#create pure fluid EOS for both fluids
NH3_0300 = EOS("NH3_0300", NH3, NH3_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0300())
H2_0313 = EOS("H2_0313", H2, H2_cp0_NIST, AlphaRHelmholtz, alpha_r_expr=alpha_r_0313())

############################################## EOS for pure mixture
#create mixture EOS
 
# mix = MixtureFluid(H2, NH3)
mix = MixtureFluid(NH3, H2)

# mixture cofficients from [0301]
n_ij = [0, -3.73558, -7.47092, 1.98413, 1.87191]
t_ij = [0, 1.28, 2.05, 2.6, 3.13]
d_ij = [0, 1, 2, 1, 2]

eta_ij = [0, 0, 0, 0.61, 1.6]
beta_ij = [0, 0, 0, 2.06, 1.74]
gamma_ij = [0, 0, 0, 0.79, 2.1]
epsilon_ij = [0, 0, 0, 0.8, 1.62]

beta_T = [[1, 0.98824],[1/0.98824, 1]]  # because [0301] take ammonia to be the first component while I use hydrogen
gamma_T = [[1,1.1266], [1.1266,1]]
beta_V = [[1, 1.0103],[1/1.0103, 1]]    # because [0301] take ammonia to be the first component while I use hydrogen
gamma_V = [[1,0.7298], [0.7298,1]]

mix_EOS = EOSMixture("H2_0313_NH3_0300", mix, AlphaRHelmholtz, NH3_0300, H2_0313,
                     F=0.1, n_ij=n_ij, t_ij=t_ij, d_ij=d_ij, eta_ij=eta_ij, beta_ij=beta_ij, gamma_ij=gamma_ij, epsilon_ij=epsilon_ij,
                     beta_T=beta_T, beta_V=beta_V, gamma_T=gamma_T, gamma_V=gamma_V)

####################################### Temperature and pressure and composition settings
temperature = 378

pressure = 50 * 101325
z = [1-0.001,0.001]

####################################### density
densities = multi_root(mix_EOS.pressure_equation, [1,5], (pressure, temperature, *z), n=1e5)


#pick a value of density
density = densities[-1]
print(f"all densities = {densities}")
print(f"density = {density} [mol/m3]")
print(f"density = {density*mix.M.subs([(z1, z[0]), (z2, z[1])])} [kg/m3]")

print("")

#density of ammonia only
densities_ammonia = multi_root(NH3_0300.pressure_equation, [1,5], (pressure, temperature), n=1e4)

density_ammonia = densities_ammonia[-1]
print(f"all densities of ammmonia = {densities_ammonia}")
print(f"density ammonia = {density_ammonia} [mol/m3]")
print(f"density ammonia = {density_ammonia*NH3.M} [kg/m3]")

print("")
# ####################################### fugacity
phi_i = mix_EOS.fugacity_coefficients

fugacity_coefficients = [(phi_i[i].subs([(z1, z[0]), (z2, z[1]), (T, temperature), (rho, density)])) for i in [0,1]]

print(f"fugacity coefficients = {fugacity_coefficients}")


####################################### Experimental data from [0511]
# The following data corresponds to the vapour state, and the first index should be used for density

# | temperature (C) | Pressure (atm) | Z H2  | Z NH3 | Fugacity coefficient of ammonia |
# | --------------- | -------------- | ----- | ----- | ------------------------------- |
# | 55.4            | 96             | 0.849 | 0.151 | 0.776                           |
# | 80.9            | 34.49          | 0     | 1     | 0.816                           |
# | 55.4            | 35.81          | 0.653 | 0.347 | 0.863                           |
# | 80.9            | 39.1           | 0.09  | 0.91  | 0.794                           |
# | 55.3            | 11.47          | 0     | 1     | 0.917                           |


# all densities = [  463.71676596  3598.52398533 13393.42019896 20389.39615436
#  36445.03632793]
# density = 36445.036327934664 [mol/m3]
# density = 620.130710983367 [kg/m3]

all densities = [ 2181.75529779  9498.08281136 13302.03120261 18965.83676016
 25852.34605204]
density = 25852.34605204167 [mol/m3]
density = 439.890732817090 [kg/m3]

all densities of ammmonia = [ 2185.83514521  9507.81040869 13270.6524488  19171.23261158
 25207.98790262]
density ammonia = 25207.98790261795 [mol/m3]
density ammonia = 429.30514213529307 [kg/m3]

fugacity coefficients = [0.949669663069958, 0.0221127677220787]
